In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [1]:
!git clone https://github.com/vslvskyy/tts_fastspeech2
%cd tts_fastspeech2

Cloning into 'tts_fastspeech2'...
remote: Enumerating objects: 148, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 148 (delta 63), reused 115 (delta 38), pack-reused 0
Receiving objects: 100% (148/148), 33.72 KiB | 4.21 MiB/s, done.
Resolving deltas: 100% (63/63), done.
/content/tts_fastspeech2


In [3]:
!pip install -r requirements.txt

In [7]:
# download LjSpeech
!wget https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2 -o /dev/null
!mkdir data
!tar -xvf LJSpeech-1.1.tar.bz2 >> /dev/null
!mv LJSpeech-1.1 data/LJSpeech-1.1

!gdown https://drive.google.com/u/0/uc?id=1-EdH0t0loc6vPiuVtXdhsDtzygWNSNZx
!mv train.txt data/

In [ ]:
# download Waveglow
!wget --load-cookies /tmp/cookies.txt \
"https://docs.google.com/uc?export=download&confirm= \
$(wget --quiet --save-cookies /tmp/cookies.txt \
--keep-session-cookies --no-check-certificate \
'https://docs.google.com/uc?export=download&id=1MDLbGuUDNEs8p96fc6IPJYwqv1-1hbrM' \
-O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1MDLbGuUDNEs8p96fc6IPJYwqv1-1hbrM" \
-O waveglow_256channels_ljs_v2.pt \
&& rm -rf /tmp/cookies.txt

!mkdir -p waveglow/pretrained_model/
!mv waveglow_256channels_ljs_v2.pt waveglow/pretrained_model/waveglow_256channels.pt

In [ ]:
# download target_melspecs
!wget --load-cookies /tmp/cookies.txt \
"https://docs.google.com/uc?export=download&confirm= \
$(wget --quiet --save-cookies /tmp/cookies.txt \
--keep-session-cookies --no-check-certificate \
'https://docs.google.com/uc?export=download&id=1YsevZHyLRf6VBQHg-dZJnePn7AwcScs3' \
-O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1YsevZHyLRf6VBQHg-dZJnePn7AwcScs3" \
-O mel.tar.gz \
&& rm -rf /tmp/cookies.txt

!tar -xvf mel.tar.gz
!echo $(ls mels | wc -l)

In [ ]:
#download alignments
!wget https://github.com/xcmyz/FastSpeech/raw/master/alignments.zip
!unzip alignments.zip >> /dev/null

In [ ]:
# download pitch_ground_truth
!wget --load-cookies /tmp/cookies.txt \
"https://docs.google.com/uc?export=download&confirm= \
$(wget --quiet --save-cookies /tmp/cookies.txt \
--keep-session-cookies --no-check-certificate \
'https://docs.google.com/uc?export=download&id=19M2wZQ30blxyDU4zpYOtBSLnfaTQAsDV' \
-O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=19M2wZQ30blxyDU4zpYOtBSLnfaTQAsDV" \
-O ljspeech_preprocessed_pitches.npy \
&& rm -rf /tmp/cookies.txt

In [ ]:
# download energy_ground_truth
!wget --load-cookies /tmp/cookies.txt \
"https://docs.google.com/uc?export=download&confirm= \
$(wget --quiet --save-cookies /tmp/cookies.txt \
--keep-session-cookies --no-check-certificate \
'https://docs.google.com/uc?export=download&id=1H9KVvXmEWw2Hpj6IcJ6XnfHZNixNdhiU' \
-O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1H9KVvXmEWw2Hpj6IcJ6XnfHZNixNdhiU" \
-O ljspeech_energies.npy \
&& rm -rf /tmp/cookies.txt

In [3]:
from configs import MelSpectrogramConfig, FastSpeechConfig, TrainConfig


mel_config = MelSpectrogramConfig()
model_config = FastSpeechConfig()
train_config = TrainConfig()

In [7]:
from utils import get_data_to_buffer


buffer = get_data_to_buffer(train_config)

100%|██████████| 13100/13100 [00:47<00:00, 273.25it/s]


cost 47.95s to load all data into buffer.


In [8]:
from torch.utils.data import DataLoader
from dataset import BufferDataset
from utils import collate_fn_tensor


dataset = BufferDataset(buffer)

training_loader = DataLoader(
    dataset,
    batch_size=train_config.batch_expand_size * train_config.batch_size,
    shuffle=True,
    collate_fn=collate_fn_tensor(train_config),
    drop_last=True,
    num_workers=0
)

In [9]:
batch = next(iter(training_loader))[0]
batch['text'].shape, batch["pitch"].shape, batch["energy"].shape, batch["mel_target"].shape, batch["mel_max_len"]

(torch.Size([16, 182]),
 torch.Size([16, 865]),
 torch.Size([16, 865]),
 torch.Size([16, 865, 80]),
 865)

In [10]:
import torch

from torch.optim.lr_scheduler  import OneCycleLR

from model import FastSpeech2
from loss import FastSpeech2Loss
from utils import get_WaveGlow

model = FastSpeech2(model_config, train_config, mel_config)
model = model.to(train_config.device)

fastspeech_loss = FastSpeech2Loss()
current_step = 0

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=train_config.learning_rate,
    betas=(0.9, 0.98),
    eps=1e-9)

scheduler = OneCycleLR(optimizer, **{
    "steps_per_epoch": len(training_loader) * train_config.batch_expand_size,
    "epochs": train_config.epochs,
    "anneal_strategy": "cos",
    "max_lr": train_config.learning_rate,
    "pct_start": 0.1
})

WaveGlow = get_WaveGlow()
WaveGlow = WaveGlow.cuda()

In [11]:
from wandb_writer import WanDBWriter
logger = WanDBWriter(train_config)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vslvskyy. Use `wandb login --relogin` to force relogin


In [ ]:
import torch.nn as nn
from tqdm import tqdm
from waveglow import get_wav
from synthesis import synthesize, get_data


tqdm_bar = tqdm(total=train_config.epochs * len(training_loader) * train_config.batch_expand_size - current_step)

for epoch in range(train_config.epochs):
    for i, batchs in enumerate(training_loader):
        # real batch start here
        for j, db in enumerate(batchs):
            current_step += 1
            tqdm_bar.update(1)
            
            logger.set_step(current_step)

            # Get Data
            character = db["text"].long().to(train_config.device)
            mel_target = db["mel_target"].float().to(train_config.device)
            pitch_target = db["pitch"].float().to(train_config.device)
            energy_target = db["energy"].float().to(train_config.device)
            duration = db["duration"].int().to(train_config.device)
            mel_pos = db["mel_pos"].long().to(train_config.device)
            src_pos = db["src_pos"].long().to(train_config.device)
            max_mel_len = db["mel_max_len"]

            # Forward
            (
                mel_output, 
                duration_predictor_output, 
                pitch_predictor_output,
                energy_predictor_output
            ) = model(
                character,
                src_pos,
                mel_pos=mel_pos,
                mel_max_length=max_mel_len,
                length_target=duration,
                pitch_target=pitch_target,
                energy_target=energy_target
            )

            # Calc Loss
            (
                mel_loss, 
                duration_loss,
                pitch_loss,
                energy_loss
            ) = fastspeech_loss(
                mel_output,
                duration_predictor_output,
                pitch_predictor_output,
                energy_predictor_output,
                mel_target,
                duration,
                pitch_target,
                energy_target
            )
            total_loss = mel_loss + duration_loss + pitch_loss + energy_loss

            # Logger
            t_l = total_loss.detach().cpu().numpy()
            m_l = mel_loss.detach().cpu().numpy()
            d_l = duration_loss.detach().cpu().numpy()
            p_l = pitch_loss.detach().cpu().numpy()
            e_l = energy_loss.detach().cpu().numpy()

            logger.add_scalar("duration_loss", d_l)
            logger.add_scalar("mel_loss", m_l)
            logger.add_scalar("total_loss", t_l)
            logger.add_scalar("pitch_loss", p_l)
            logger.add_scalar("energy_loss", e_l)

            # Backward
            total_loss.backward()

            # Clipping gradients to avoid gradient explosion
            nn.utils.clip_grad_norm_(
                model.parameters(), train_config.grad_clip_thresh)
            
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()

            if current_step % (train_config.save_step // 6) == 0:
                model.eval()
                inp = [
                    "A defibrillator is a device that gives a high energy electric \
                                        shock to the heart of someone who is in cardiac arrest"
                ]
                mel_spec = synthesize(
                    model, get_data(inp, train_config)[0], train_config.device, 1.0, 1.0, 1.0
                )
                audio = get_wav(mel_spec, WaveGlow)
                logger.add_audio("sample wav", audio, sample_rate=train_config.sample_rate)
                model.train()

            if current_step % train_config.save_step == 0:
                torch.save({'model': model.state_dict(), 'optimizer': optimizer.state_dict(
                )}, '/content/drive/MyDrive/tts(dla_hw)/checkpoint_%d.pth.tar' % current_step)
                print("save model at step %d ..." % current_step)


In [13]:
import wandb

wandb.finish()

train/duration_loss,█▄▄▃▄▃▃▃▄▁▂▂▃▂▂▂▂▂▃▂▂▂▂▁▂▂▁▂▁▂▁▁▁▂▂▂▁▁▁▁
train/energy_loss,▄▆█▅▃▆▅▂▄▂▂▆▃▃▇▂▃▃▄▂▃▄▆▃▅▃▅▃▃▂▃▁▂▂▄▅▂▃▂▁
train/mel_loss,█▆▅▄▄▃▃▂▂▂▂▂▂▂▃▂▁▂▂▁▂▂▂▂▁▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
train/pitch_loss,█▇█▅▅▄▅▃▃▂▄▄▄▄▄▂▃▂▃▂▂▄▃▃▂▃▃▃▃▂▁▁▂▃▂▂▂▂▂▁
train/steps_per_sec,▇▇▇▁▆▅▆▆▆▆▅▇▇▇▆▇▆▆▆▅▆▇▇▇▇▇▆▇▇▆▆▇▇▆▆▆█▇▆▇
train/total_loss,█▇▇▅▄▄▄▃▃▂▄▄▃▃▄▂▂▂▃▂▂▃▃▃▂▃▃▃▃▂▁▁▂▂▂▂▂▁▂▁
train/duration_loss,0.19856
train/energy_loss,0.16989
train/mel_loss,0.22853
train/pitch_loss,1.01118
train/steps_per_sec,3.46571


In [15]:
model = model.eval()

inp = [
    "A defibrillator is a device that gives a high energy electric shock to the heart of someone who is in cardiac arrest"
]

speed, pitch, energy = 1., 1., 1.
mel_spec = synthesize(
    model, get_data(inp, train_config)[0], train_config.device, speed, pitch, energy
)

In [16]:
from IPython import display
from waveglow import inference


inference(
    mel_spec, WaveGlow,
    f"./s={speed}_p={pitch}_e={energy}_waveglow.wav"
)

display.Audio(f"./s=1.0_p=1.0_e=1.0_waveglow.wav")